# 2D Advection-Diffusion equation

in this notebook we provide a simple example of the DeepMoD algorithm and apply it on the 2D advection-diffusion equation. 

In [8]:
# General imports
import numpy as np
import torch
# DeepMoD functions
from deepymod import DeepMoD
from deepymod.model.func_approx import NN
from deepymod.model.library import Library2D_third
from deepymod.model.constraint import LeastSquares
from deepymod.model.sparse_estimators import Threshold,PDEFIND
from deepymod.training import train
from deepymod.training.sparsity_scheduler import TrainTestPeriodic
from scipy.io import loadmat

from sklearn.linear_model import LassoCV
from derivatives import finite_diff


# Settings for reproducibility
np.random.seed(1)
torch.manual_seed(1)

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

## Prepare the data

Next, we prepare the dataset.

In [3]:
data_pre = np.load('diffusion_advection_29longb.npy').T
data = data_pre[120:-60,:,5:30]
x_dim, y_dim, t_dim = data.shape

In [13]:
# Prepare data
idx, idt = 1 , 1
down_data= np.take(np.take(np.take(data,np.arange(0,x_dim,idx),axis=0),np.arange(0,y_dim,idx),axis=1),np.arange(0,t_dim,idt),axis=2)
print("Dowmsampled shape:",down_data.shape)
width, width_2, steps = down_data.shape
x_arr, y_arr, t_arr = np.linspace(0,1,width), np.linspace(0,1,width_2), np.linspace(0,1,steps)
x_grid, y_grid, t_grid = np.meshgrid(x_arr, y_arr, t_arr, indexing='ij')
X = np.transpose((t_grid.flatten(), x_grid.flatten(), y_grid.flatten()))
y = down_data.reshape((down_data.size, 1))

# Add noise to the solution 
noise_level = 0.0
y_noisy = y + noise_level * np.std(y) * np.random.randn(y.size, 1)
solution = y_noisy.reshape(down_data.shape)

dim_w = 3
denoised_sol = []
for i in np.arange(steps):
    uwn,sigmawn,vwn= np.linalg.svd(solution[:,:,i])
    vwn = vwn.T
    denoised_sol.append(uwn[:,0:dim_w].dot(np.diag(sigmawn[0:dim_w]).dot(vwn[:,0:dim_w].T)))
denoised_sol = np.transpose(np.array(denoised_sol),(1,2,0))


Dowmsampled shape: (120, 150, 25)


In [14]:
mask = np.array([False,False,True,True,True,False])
ref = np.array([0,0,0.2,0.02,0.02,0]) 
ref_b =  np.array([0,0,0.2,0.02,0.02,0]) + 10**-9*np.ones(6)

In [15]:
bor = 2
# Construct the library
u = denoised_sol[:,:,:][bor:-bor,bor:-bor,:]
u_t = finite_diff(denoised_sol[:,:,:],t_arr,order=1,axis=2)[bor:-bor,bor:-bor,:].reshape((-1, 1))
u_x = finite_diff(denoised_sol[:,:,:],x_arr,order=1,axis=0)[bor:-bor,bor:-bor,:]
u_xx = finite_diff(denoised_sol[:,:,:],x_arr,order=2,axis=0)[bor:-bor,bor:-bor,:]
u_y = finite_diff(denoised_sol[:,:,:],y_arr,order=1,axis=1)[bor:-bor,bor:-bor,:]
u_yy = finite_diff(denoised_sol[:,:,:],y_arr,order=2,axis=1)[bor:-bor,bor:-bor,:]
u_xy = finite_diff(finite_diff(denoised_sol[:,:,:],x_arr,order=1,axis=0),y_arr,order=1,axis=1)[bor:-bor,bor:-bor,:]

library = np.transpose((np.ones_like(u).flatten(), u_x.flatten(), u_y.flatten(),
                  u_xx.flatten(),u_yy.flatten(),u_xy.flatten()))

[ 0.00000000e+00  4.00104104e-01 -9.89956007e-03  5.92485650e-05
  1.12730342e-04 -7.46509247e-05]
wrong


In [16]:
reg = LassoCV(fit_intercept=False, cv=10)
xi = reg.fit(library, u_t).coef_[:, None].flatten()
print(xi)
if np.ma.allequal(xi.astype(bool), mask) == True:
    print('correct')
    k = np.sum(np.abs((xi-ref)/ref_b))
    print(np.sum(np.abs((xi-ref)/ref_b)))
else: 
    print('wrong')

[ 0.00000000e+00  4.00104104e-01 -9.89956007e-03  5.92485650e-05
  1.12730342e-04 -7.46509247e-05]
wrong
